In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('notifs_corrected.csv')

In [6]:
oneh = np.stack(one_hotted,axis=0)

In [3]:
from tqdm import tqdm
def f(x):
    if isinstance(x,str) == False:
        return None
    else:
        return np.array(x.split(' '))
text_series = data['text'].apply(lambda x : f(x))
text_series = text_series[text_series.notna()]
def g(x):
    arr = []
    l = len(x)
    for i in range(67):
        p = np.zeros(2000)
        if i<l:
            p[int(x[i])] = 1
        else:
            p[1999]=1
        arr.append(p)
    return np.array(arr)
one_hotted = text_series.apply(lambda x:g(x))

In [9]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [70]:
!pip install pydot

In [10]:
# configure
num_encoder_tokens = 2000
num_decoder_tokens = 2000
latent_dim = 32

In [11]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [12]:
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [14]:
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
# plot the model
#badan plot kon
# plot_model(model, to_file='model.png', show_shapes=True)
# define encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

In [17]:
# define decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [80]:
# summarize model
# plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
# plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)

In [1]:
oneh

NameError: name 'oneh' is not defined

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit([oneh,oneh], batch_size=32, epochs=1, verbose=1)

IndexError: list index out of range